Import Libraries And Constants

In [10]:
folder = r"C:\Users\franc\Desktop\Usach Proyecto Test"
import sys
sys.path.append("..")
from Models.BlockModelDilution import BlockModelDilution

from Models.utils import best_height_of_draw, sequence_footprint
import Models.Factory as ft
import Models.visualization as vs
from Scripts.ValorizationScript import valorization
import Models.visualization as visualization
from Engine.CavingProductionPlanExtractionSpeedItem import CavingProductionPlanExtractionSpeedItem
from Engine.CavingProductionPlanTargetItem import CavingProductionPlanTargetItem
from Engine.CavingProductionPlanTarget import CavingProductionPlanTarget
from Engine.ProductionPlanEngine import ProductionPlanEngine
from Scripts.Optimizer import build_inclined_footprints, compute_tonnage_grade, draw_grades, compute_best_height_level, compute_best_height, generate_plan, segment_squares, stats_footprints
from Scripts import ValorizationScript
from Models.utils import best_height_of_draw,get_average,get_summation,get_average_from_footprint,get_summation_from_footprint
from Models.excel_utils import export_matrix_to_excel


import numpy as np 

from os import path
    




In [12]:
# -- Constants --
folder = r"C:\Users\franc\Desktop\Usach Proyecto Test"
level = 7
csv_name = path.join(folder,"G8.csv")
npy_name = path.join(folder,"G8.npy")
cut_name = "Cut"
density_name = "Density"
recovery_name = "Recu"
diluted_bm_path = path.join(folder,f"Diluted BM {level}.npy")
footprint_path = path.join(folder,f"Footprint {level}.xlsx")
sequence_path = path.join(folder,f"Sequence {level}.xlsx")
plan_configuration_path = path.join(folder,f"Configuration Plan.xlsx")
value_accumulated_set = 'accumulated_value'
value_set = 'value'
ped = 40 # Percentage




In [3]:
import pyvista as pv
from pyvista import examples
from pyvista import demos

mb_path_csv = path.join(folder,csv_name)
mb_path_np = path.join(folder,npy_name)

block_model_insitu = ft.block_model_from_csv_file(mb_path_csv,'X','Y','Z')
visualization.contours(block_model_insitu,cut_name,[0.8])
recu = block_model_insitu.get_data_set(recovery_name)
cut = block_model_insitu.get_data_set(cut_name)

# Clean invalid values
recu[recu<0] = 60
block_model_insitu.update_dataset(recovery_name,recu)
block_model_insitu.save(mb_path_np)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Dilución del Modelo de bloques

In [4]:
 # Crear una calculadora de dilución
 

block_model_insitu = ft.block_model_from_npy_file(path.join(folder,npy_name))
level_block_model = ft.block_model_from_level(block_model_insitu, level)  # Get the 7th level of the block model


block_dilution_engine = BlockModelDilution(ped, level_block_model.structure)
block_dilution_engine.compute_dilution_coefficients()

#  Dilute all the data sets
block_model_diluted = level_block_model.clone()
for data_set_name in level_block_model.get_dataset_names():
    data_set = level_block_model.get_data_set(data_set_name)
    diluted_data_set = block_dilution_engine.dilute_dataset(data_set)
    block_model_diluted.update_dataset(data_set_name, diluted_data_set)
    

ValorizationScript.valorization(block_model_diluted)
block_model_diluted.save(diluted_bm_path)


In [5]:
block_model_diluted = ft.block_model_from_npy_file(diluted_bm_path)
footprint = best_height_of_draw(block_model_diluted,value_accumulated_set)
#footprint.export_to_excel(footprint_path)

In [8]:
block_model_diluted = ft.block_model_from_npy_file(diluted_bm_path)

footprint = ft.footprint_from_excel(footprint_path,block_model_diluted)
sequence = sequence_footprint(footprint,30)
sequence.export_to_excel(sequence_path)
#visualization.draw_footprint(footprint,block_model_diluted,footprint.compute_slope_percentage())

average = get_average_from_footprint(footprint,block_model_diluted,cut_name,density_name)
summation = get_summation_from_footprint(footprint,block_model_diluted,cut_name)
export_matrix_to_excel(average,path.join(folder,"Average.xlsx"))
export_matrix_to_excel(summation,path.join(folder,"Summation.xlsx"))








In [16]:
# Target
first_item_target = CavingProductionPlanTargetItem(1, 1e6, 10, 360)
target_items: list[CavingProductionPlanTargetItem] = [first_item_target]

# Speed
first_item_speed = CavingProductionPlanExtractionSpeedItem(0, 30, 0.15)
speed_items: list[CavingProductionPlanExtractionSpeedItem] = [first_item_speed]

# Caving plan

# configuration = ft.CavingProductionPlanTarget("configuration",density_name,target_items,speed_items,[cut_name],[value_set])
# configuration.export_to_excel(plan_configuration_path)
configuration = ft.caving_configuration_from_excel(plan_configuration_path)

productionPlanEngine = ProductionPlanEngine(block_model_diluted, footprint,sequence, configuration)
result = productionPlanEngine.process()
path_to_excel_caving_plan_result =path.join(folder,"Plan.xlsx")
result.export_excel(path_to_excel_caving_plan_result)